# Multilayer perceptron

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline 

See MIT License in 
https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline/README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 [Online.](https://arxiv.org/abs/1611.06455 "Wang et al. (2017)")

## Data
Additional datasets are available at http://www.timeseriesclassification.com/. Save additional datasets in deepscent/data. E.g. deepscent/data/Adiac/Adiac_TRAIN.txt

## Formatting
Left align the tables in this Notebook.

In [1]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

## Train MLP
Expected running time for various datasets, on a single NVIDIA GeForce GTX 1080 Ti Graphics Card -

Dataset   | Training time
:-------  | :--------
Adiac     | 18 minutes
GunPoint  | 11 minutes


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import utils
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd

import os
import time
from datetime import datetime
import pathlib

np.random.seed(813306)

      
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 5000

# Directories
fdir = '../data'  
logs_dir = '../logs'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now())
logs_dir = logs_dir +'/' + timestamp
#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'GunPoint', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist = ['GunPoint']
for each in flist:
    fname = each
    print('Running dataset', fname)
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    nb_classes =len(np.unique(y_test))
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    batch_size = int(min(x_train.shape[0]/10, 16))
    
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
   # x_test_min = np.min(x_test, axis = 1, keepdims=1)
   # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean)/(x_train_std)
     
    #x_train = x_train.reshape(x_train.shape + (1,))
    #x_test = x_test.reshape(x_test.shape + (1,))
    
    x = Input(x_train.shape[1:])
    y= Dropout(0.1)(x)
    y = Dense(500, activation='relu')(x)
    y = Dropout(0.2)(y)
    y = Dense(500, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(500, activation = 'relu')(y)
    y = Dropout(0.3)(y)
    out = Dense(nb_classes, activation='softmax')(y)
     
    model = Model(x, out)
     
    optimizer = keras.optimizers.Adadelta(rho=0.95, epsilon=1e-8)    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=200, min_lr=0.1)
    

    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), 
                #callbacks = [TestCallback((x_train, Y_train)), reduce_lr, keras.callbacks.TensorBoard(log_dir='./log'+fname, histogram_freq=1)])
                 callbacks=[reduce_lr])
    end = time.time()
    
    log = pd.DataFrame(hist.history)
    # Print results. Print the testing results which has the lowest training loss.
    print('Training complete on', fname)
    print('Training time ', end-start, 'seconds, which is about', round((end-start)/60), 'minutes.')
    print('Test result with the lowest training loss. Loss and accuracy are -')

    loss = log.loc[log['loss'].idxmin]['loss']
    val_acc = log.loc[log['loss'].idxmin]['val_acc']
    print(loss, val_acc)
    print('For the results table -')
    print('|#   |',loss,'  |',val_acc,' |')
    # Save log file.
    pathlib.Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True) 
    print('Saving logs to',logs_dir+'/'+fname+'/history.csv')
    log.to_csv(logs_dir+'/'+fname+'/history.csv')
 



Running dataset GunPoint
Train on 50 samples, validate on 150 samples
Epoch 1/5000
50/50 [==============================] - 0s 10ms/step - loss: 0.7234 - acc: 0.5000 - val_loss: 0.5722 - val_acc: 0.6933
Epoch 2/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.5345 - acc: 0.7000 - val_loss: 0.4682 - val_acc: 0.7467
Epoch 3/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.4735 - acc: 0.7200 - val_loss: 0.5158 - val_acc: 0.6933
Epoch 4/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.4178 - acc: 0.8400 - val_loss: 0.5282 - val_acc: 0.7400
Epoch 5/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.2182 - acc: 0.9600 - val_loss: 0.4150 - val_acc: 0.7933
Epoch 6/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.4268 - acc: 0.8000 - val_loss: 0.4058 - val_acc: 0.7867
Epoch 7/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.2794 - acc: 0.7600 - val_loss: 0.4689 - val_acc: 0.7667
Epoch

50/50 [==============================] - 0s 3ms/step - loss: 0.0100 - acc: 1.0000 - val_loss: 0.2985 - val_acc: 0.9000
Epoch 63/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.0151 - acc: 1.0000 - val_loss: 0.2636 - val_acc: 0.9200
Epoch 64/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0225 - acc: 1.0000 - val_loss: 0.2427 - val_acc: 0.9400
Epoch 65/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0117 - acc: 1.0000 - val_loss: 0.2968 - val_acc: 0.9133
Epoch 66/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0339 - acc: 0.9800 - val_loss: 0.3109 - val_acc: 0.9000
Epoch 67/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.0465 - acc: 0.9800 - val_loss: 0.5527 - val_acc: 0.8133
Epoch 68/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.1416 - acc: 0.9200 - val_loss: 0.2591 - val_acc: 0.9267
Epoch 69/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0330 -

50/50 [==============================] - 0s 3ms/step - loss: 6.2544e-04 - acc: 1.0000 - val_loss: 0.3894 - val_acc: 0.9267
Epoch 124/5000
50/50 [==============================] - 0s 2ms/step - loss: 6.8771e-04 - acc: 1.0000 - val_loss: 0.3837 - val_acc: 0.9333
Epoch 125/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.6416e-04 - acc: 1.0000 - val_loss: 0.3826 - val_acc: 0.9400
Epoch 126/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.0042 - acc: 1.0000 - val_loss: 0.4707 - val_acc: 0.8933
Epoch 127/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.3981 - val_acc: 0.9400
Epoch 128/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.4366 - val_acc: 0.9267
Epoch 129/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0072 - acc: 1.0000 - val_loss: 0.6165 - val_acc: 0.8933
Epoch 130/5000
50/50 [==============================] - 0s 2ms/st

Epoch 183/5000
50/50 [==============================] - 0s 2ms/step - loss: 3.9805e-05 - acc: 1.0000 - val_loss: 0.4663 - val_acc: 0.9333
Epoch 184/5000
50/50 [==============================] - 0s 2ms/step - loss: 3.8916e-04 - acc: 1.0000 - val_loss: 0.4985 - val_acc: 0.9333
Epoch 185/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.1037e-04 - acc: 1.0000 - val_loss: 0.5057 - val_acc: 0.9267
Epoch 186/5000
50/50 [==============================] - 0s 3ms/step - loss: 3.6898e-05 - acc: 1.0000 - val_loss: 0.4990 - val_acc: 0.9333
Epoch 187/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.0040e-04 - acc: 1.0000 - val_loss: 0.4897 - val_acc: 0.9400
Epoch 188/5000
50/50 [==============================] - 0s 3ms/step - loss: 7.6395e-05 - acc: 1.0000 - val_loss: 0.4784 - val_acc: 0.9467
Epoch 189/5000
50/50 [==============================] - 0s 2ms/step - loss: 1.5204e-04 - acc: 1.0000 - val_loss: 0.5039 - val_acc: 0.9267
Epoch 190/5000
50/50 [============

50/50 [==============================] - 0s 2ms/step - loss: 2.1505e-04 - acc: 1.0000 - val_loss: 0.7032 - val_acc: 0.9067
Epoch 243/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.0297e-05 - acc: 1.0000 - val_loss: 0.5919 - val_acc: 0.9333
Epoch 244/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.5795e-06 - acc: 1.0000 - val_loss: 0.5830 - val_acc: 0.9333
Epoch 245/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.2481e-05 - acc: 1.0000 - val_loss: 0.5896 - val_acc: 0.9333
Epoch 246/5000
50/50 [==============================] - 0s 2ms/step - loss: 2.0322e-05 - acc: 1.0000 - val_loss: 0.5586 - val_acc: 0.9400
Epoch 247/5000
50/50 [==============================] - 0s 3ms/step - loss: 3.7501e-05 - acc: 1.0000 - val_loss: 0.5356 - val_acc: 0.9400
Epoch 248/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.6993e-06 - acc: 1.0000 - val_loss: 0.5366 - val_acc: 0.9400
Epoch 249/5000
50/50 [===========================

...

50/50 [==============================] - 0s 3ms/step - loss: 1.8275e-06 - acc: 1.0000 - val_loss: 0.5464 - val_acc: 0.9333
Epoch 833/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.5463e-06 - acc: 1.0000 - val_loss: 0.5459 - val_acc: 0.9333
Epoch 834/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.0777e-06 - acc: 1.0000 - val_loss: 0.5459 - val_acc: 0.9333
Epoch 835/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.3304e-06 - acc: 1.0000 - val_loss: 0.5462 - val_acc: 0.9333
Epoch 836/5000
50/50 [==============================] - 0s 2ms/step - loss: 4.7207e-07 - acc: 1.0000 - val_loss: 0.5462 - val_acc: 0.9333
Epoch 837/5000
50/50 [==============================] - 0s 3ms/step - loss: 3.2902e-07 - acc: 1.0000 - val_loss: 0.5462 - val_acc: 0.9333
Epoch 838/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.3020e-06 - acc: 1.0000 - val_loss: 0.5461 - val_acc: 0.9333
Epoch 839/5000
50/50 [===========================

50/50 [==============================] - 0s 2ms/step - loss: 1.3578e-06 - acc: 1.0000 - val_loss: 0.5486 - val_acc: 0.9400
Epoch 892/5000
50/50 [==============================] - 0s 2ms/step - loss: 1.5664e-06 - acc: 1.0000 - val_loss: 0.5491 - val_acc: 0.9400
Epoch 893/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.1909e-06 - acc: 1.0000 - val_loss: 0.5486 - val_acc: 0.9400
Epoch 894/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.7936e-07 - acc: 1.0000 - val_loss: 0.5487 - val_acc: 0.9400
Epoch 895/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.0705e-06 - acc: 1.0000 - val_loss: 0.5483 - val_acc: 0.9333
Epoch 896/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.4952e-06 - acc: 1.0000 - val_loss: 0.5498 - val_acc: 0.9400
Epoch 897/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.2672e-06 - acc: 1.0000 - val_loss: 0.5493 - val_acc: 0.9400
Epoch 898/5000
50/50 [===========================

50/50 [==============================] - 0s 2ms/step - loss: 2.3079e-06 - acc: 1.0000 - val_loss: 0.5532 - val_acc: 0.9400
Epoch 951/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.5005e-06 - acc: 1.0000 - val_loss: 0.5528 - val_acc: 0.9400
Epoch 952/5000
50/50 [==============================] - 0s 2ms/step - loss: 6.8903e-07 - acc: 1.0000 - val_loss: 0.5526 - val_acc: 0.9400
Epoch 953/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.4520e-06 - acc: 1.0000 - val_loss: 0.5520 - val_acc: 0.9333
Epoch 954/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.7014e-06 - acc: 1.0000 - val_loss: 0.5507 - val_acc: 0.9333
Epoch 955/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.6189e-06 - acc: 1.0000 - val_loss: 0.5507 - val_acc: 0.9333
Epoch 956/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.9312e-06 - acc: 1.0000 - val_loss: 0.5502 - val_acc: 0.9333
Epoch 957/5000
50/50 [===========================

50/50 [==============================] - 0s 2ms/step - loss: 6.0082e-07 - acc: 1.0000 - val_loss: 0.5542 - val_acc: 0.9400
Epoch 1010/5000
50/50 [==============================] - 0s 2ms/step - loss: 3.6623e-06 - acc: 1.0000 - val_loss: 0.5528 - val_acc: 0.9400
Epoch 1011/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.2982e-06 - acc: 1.0000 - val_loss: 0.5527 - val_acc: 0.9400
Epoch 1012/5000
50/50 [==============================] - 0s 3ms/step - loss: 7.0811e-07 - acc: 1.0000 - val_loss: 0.5527 - val_acc: 0.9400
Epoch 1013/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.2943e-07 - acc: 1.0000 - val_loss: 0.5524 - val_acc: 0.9400
Epoch 1014/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.8007e-05 - acc: 1.0000 - val_loss: 0.5607 - val_acc: 0.9333
Epoch 1015/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.3530e-06 - acc: 1.0000 - val_loss: 0.5607 - val_acc: 0.9333
Epoch 1016/5000
50/50 [====================

50/50 [==============================] - 0s 3ms/step - loss: 2.3056e-06 - acc: 1.0000 - val_loss: 0.5611 - val_acc: 0.9333
Epoch 1069/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.2429e-06 - acc: 1.0000 - val_loss: 0.5590 - val_acc: 0.9400
Epoch 1070/5000
50/50 [==============================] - 0s 2ms/step - loss: 6.6750e-06 - acc: 1.0000 - val_loss: 0.5585 - val_acc: 0.9400
Epoch 1071/5000
50/50 [==============================] - 0s 2ms/step - loss: 2.5892e-05 - acc: 1.0000 - val_loss: 0.5506 - val_acc: 0.9400
Epoch 1072/5000
50/50 [==============================] - 0s 2ms/step - loss: 2.9195e-06 - acc: 1.0000 - val_loss: 0.5501 - val_acc: 0.9400
Epoch 1073/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.2319e-07 - acc: 1.0000 - val_loss: 0.5501 - val_acc: 0.9400
Epoch 1074/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.4618e-06 - acc: 1.0000 - val_loss: 0.5491 - val_acc: 0.9400
Epoch 1075/5000
50/50 [====================

# Recorded results
## Adiac

|Run |Loss |Accuracy | Comment
|:---|:--- |:---     |:----------
|1   |0.005222544357037315  |0.6930946294608933 |Adadelta - default values
|2   |0.0018498263113997382 |0.7109974427601261 |
|3   |0.001620212956988372  |0.7212276216358176 |Adadelta parameters match Wang (2017)
|4   |0.0023210097823697976 |0.7289002559069172  |

## GunPoint

|Run |Loss |Accuracy | Comment
|:---|:--- |:---     |:----------
|1   |1.466274483163943e-07  |0.9400000035762787 |Adadelta parameters match Wang (2017)

In [ ]:
# Read results from file

In [7]:
read_log = pd.read_csv(logs_dir+'/'+fname+'/history.csv')
read_loss = read_log.loc[read_log['loss'].idxmin]['loss']
read_val_acc = read_log.loc[read_log['loss'].idxmin]['val_acc']
print('Read from file', logs_dir+'/'+fname+'/history.csv', '- loss and val_acc are')
print(read_loss, read_val_acc)

Read from file ../logs/2018-11-17T13:33/Adiac/history.csv loss and val_acc are
0.0023210097823697976 0.7289002559069172
